A Fórmula Mágica é um modelo de fator que classifica as ações por dois fatores:

1. O valor de uma empresa em relação aos seus lucros determina quão “barato” é o preço de mercado de uma ação. Greenblatt define “Barato” como o valor de uma empresa em relação aos seus lucros. Na maioria das vezes, podemos ver isto representado pelo P/L, Greenblatt prefere olhar para EV/EBIT. Isto permite que empresas com diferentes estruturas de endividamento e impostos sejam comparadas mais facilmente. O EV/EBIT compara o Valor da Empresa (Enterprise Value, EV = Capitalização de mercado + Dívida total - Caixa e equivalentes de caixa) ao seu Lucro Antes de Juros e Impostos (Earnings Before Interest and Taxes, EBIT), o EV leva em conta o valor de mercado total da empresa, adicionando as dívidas e subtraindo o caixa, oferecendo assim, uma visão mais abrangente do valor da empresa do que simplesmente seu valor de mercado. O EBIT é uma medida da capacidade de geração de lucro operacional da empresa, excluindo os efeitos das decisões de estrutura de capital e impostos. Um EV/EBIT menor sugere que o preço da ação da empresa pode estar baixo em comparação com a quantidade de lucro operacional que está gerando, indicando que a empresa está subavaliada pelo mercado. No entanto, é importante analisar o EV/EBIT dentro do contexto de cada setor, já que diferentes indústrias têm padrões de avaliação distintos.

2. O retorno sobre o capital determina o quão “boa” é uma empresa. “Bom” é representado pelo ROIC, Greenblatt quantifica a quantidade de capital tangível necessária para operar um negócio e quanto dinheiro cada unidade de capital aplicado irá render. O ROIC é um dos indicadores financeiros mais importantes para analisar a eficiência operacional de uma empresa. Para calcular o ROIC, divide-se o lucro operacional líquido de impostos pelo capital investido total, que inclui dívidas e patrimônio líquido. Um ROIC alto é geralmente interpretado como um sinal de que a empresa está utilizando seu capital de maneira eficaz para gerar lucros.

ROIC utilizada por Greenblatt é diferente da fórmula tradicional

ROIC = EBIT / (Capital Circulante Líquido + Ativos Fixos Líquidos)

Onde o Capital circulante Líquido é um proxy do Capital de Giro da Empresa (Ativo Circulante – Passivo Circulante), os Ativos Fixos Líquidos é proxy dos ativos permanentes (imobilizado e intangíveis, utilizados para gerar retorno no longo prazo. A Taxa de Retorno de Lucros para uma empresa é a fórmula tradicional: Taxa de Retorno = Lucro Operacional / Valor da Empresa

Estes dois indicadores são colocados cada um em um ranking, da melhor (rank 1) para a pior empresa. Depois somamos os números das duas colunas, quanto menor a soma, melhor.

Para os princípios de Greenblatt, empresas pertencentes aos setores de utilidades públicas, bancos e  seguradoras devem ser excluídas da seleção. 

In [3]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import requests
from IPython.display import display, HTML


def pct_to_float(number):
    """Convert string to float, remove % char and set decimal point to '.'."""
    return float(number.strip("%").replace(".", "").replace(",", "."))


def b_print(df , n=30 , clean=True):
    
    # from IPython.display import display, HTML

    if clean : # remove tickers da mesma empresa, deixando a primeria ocorrencia
        df['prefixo'] = df['Papel'].astype(str).str[:4]
        df=df.drop_duplicates(subset='prefixo', keep='first')
        # df=df.drop('prefixo', axis=1) 
    
    display(HTML(df.head(n).to_html(index=False)))
    df = None

def setor(setor:int):

    url = f'http://www.fundamentus.com.br/resultado.php?setor={setor}'
    hdr = {'User-agent': 'Mozilla/5.0 (Windows; U; Windows NT 6.1; rv:2.2) Gecko/20110201' ,
           'Accept': 'text/html, text/plain, text/css, text/sgml, */*;q=0.01' ,
           'Accept-Encoding': 'gzip, deflate' ,
           }
    content = requests.get(url, headers=hdr)
    df = pd.read_html(content.text, decimal=",", thousands='.')[0]
    return list(df['Papel'])



dados do site https://www.fundamentus.com.br/resultado.php

In [4]:
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}
url = 'https://www.fundamentus.com.br/resultado.php'
#junta com a requests
r = requests.get(url, headers=header)
# read_html do pandas põe a tabela num dataframe
funda = pd.read_html(r.text, index_col="Papel",
                     decimal=',', thousands='.',encoding='ISO-8859-1', 
                     converters={'ROE': pct_to_float,
                                 'ROIC': pct_to_float,
                                 'Div.Yield':pct_to_float,
                                 'Mrg Ebit':pct_to_float,
                                 'Mrg. Líq.':pct_to_float,
                                 'Cresc. Rec.5a':pct_to_float,
                                 },
)[0]
data_funds = pd.DataFrame(funda)
data_funds.columns

Index(['Cotação', 'P/L', 'P/VP', 'PSR', 'Div.Yield', 'P/Ativo', 'P/Cap.Giro',
       'P/EBIT', 'P/Ativ Circ.Liq', 'EV/EBIT', 'EV/EBITDA', 'Mrg Ebit',
       'Mrg. Líq.', 'Liq. Corr.', 'ROIC', 'ROE', 'Liq.2meses', 'Patrim. Líq',
       'Dív.Brut/ Patrim.', 'Cresc. Rec.5a'],
      dtype='object')

In [5]:
earning_yield = 'EV/EBIT'  #Quanto menor melhor
return_on_capital = 'ROIC' #ROIC – Quanto maior melhor

### filtros e ranking

- Empresa precisa ter ROIC positivo
- Empresa precisa ter Lucro Operacional positivo.
- Volume de negociação diário medio maior que R$ 1000000.000,00 nos últimos 2 meses.

ROIC – Quanto maior melhor

EV/EBIT – Quanto menor melhor

In [6]:
funds = data_funds.copy()

funds =  funds[funds[earning_yield] > 0]
funds =  funds[funds[return_on_capital] > 0]
funds =  funds[funds['Liq.2meses'] > 1000000]     #Volume diário médio negociado nos últimos 2 meses

""" magic formula rank."""
funds["Rank_earnings_yield"]   = funds[earning_yield].rank(ascending=True, method="min")
funds["Rank_return_on_capital"]= funds[return_on_capital].rank(ascending=False, method="min")
funds["Rank_Final"] = (funds["Rank_earnings_yield"] + funds["Rank_return_on_capital"])
funds.sort_values(by="Rank_Final", ascending=True, inplace=True)
funds.reset_index(inplace=True)
funds.index = funds.index + 1

### lista ranqueada

In [7]:
funds.columns

Index(['Papel', 'Cotação', 'P/L', 'P/VP', 'PSR', 'Div.Yield', 'P/Ativo',
       'P/Cap.Giro', 'P/EBIT', 'P/Ativ Circ.Liq', 'EV/EBIT', 'EV/EBITDA',
       'Mrg Ebit', 'Mrg. Líq.', 'Liq. Corr.', 'ROIC', 'ROE', 'Liq.2meses',
       'Patrim. Líq', 'Dív.Brut/ Patrim.', 'Cresc. Rec.5a',
       'Rank_earnings_yield', 'Rank_return_on_capital', 'Rank_Final'],
      dtype='object')

In [8]:
funds[['Papel','EV/EBITDA', 'ROIC', 'Liq.2meses','Rank_earnings_yield', 'Rank_return_on_capital', 'Rank_Final']].head(20)

,Papel,EV/EBITDA,ROIC,Liq.2meses,Rank_earnings_yield,Rank_return_on_capital,Rank_Final
1,PSSA3,0.48,84.44,8.809070e+07,1.0,1.0,2.0
2,PLPL3,2.71,44.22,1.280620e+07,6.0,2.0,8.0
3,WIZC3,1.36,31.37,2.972500e+06,2.0,6.0,8.0
4,PETR4,2.48,23.16,1.123500e+09,7.0,17.0,24.0
5,PETR3,2.65,23.16,3.685240e+08,8.0,17.0,25.0
6,CMIN3,3.64,30.62,4.243710e+07,19.0,8.0,27.0
7,SYNE3,1.52,16.74,7.587810e+06,3.0,32.0,35.0
8,VALE3,3.11,18.51,1.613950e+09,12.0,28.0,40.0
9,LEVE3,4.82,33.91,1.011840e+07,35.0,5.0,40.0
10,VLID3,3.75,20.16,1.154490e+07,23.0,22.0,45.0


retira banco, seguradora, utilidade publica (agua e luz - [util-b3](https://www.b3.com.br/pt_br/market-data-e-indices/indices/indices-de-segmentos-e-setoriais/indice-utilidade-publica-util-composicao-da-carteira.htm))


In [9]:
bancos = setor(20)
print(*bancos)
seguros = setor(31)
print(*seguros)
util_agua = setor(2)
util_luz = setor(14)
util=util_agua+util_luz
print(*util)


RPAD5 RPAD3 RPAD6 BGIP4 BGIP3 BAZA3 PINE4 PINE3 BBAS3 BNBR3 BRSR6 BMEB3 BMGB4 ABCB4 BRSR3 BMEB4 BRSR5 ITSA4 BRBI11 BBDC3 ITSA3 BEES3 BEES4 ITUB3 SANB3 BBDC4 BPAC5 SANB11 SANB4 ITUB4 BMIN4 BMIN3 BPAC11 BPAN4 BSLI4 BPAC3 MERC4 BSLI3
WIZC3 BBSE3 PSSA3 CXSE3 IRBR3
AMBP3 SAPR11 SAPR4 SAPR3 CSMG3 SBSP3 ORVR3 GPAR3 CBEE3 LIGT3 CEED3 RNEW4 RNEW3 RNEW11 CMIG4 CLSC3 CLSC4 ISAE4 EQMA3B CEEB5 CMIG3 CEEB3 EQPA3 NEOE3 ENGI4 ISAE3 COCE5 COCE3 CEBR5 CEBR3 CPFE3 EGIE3 ENGI11 CEBR6 TAEE3 TAEE11 EQPA5 TAEE4 ELET3 CPLE3 EQPA7 ALUP4 REDE3 ELET6 ALUP11 ALUP3 CPLE6 EKTR4 CPLE5 ENGI3 EMAE4 AURE3 ENMT4 ENMT3 EQTL3 ELET5 SRNA3 LIPR3 AFLT3 ENEV3 GEPA3 GEPA4


#### sem bancos

In [10]:
# funds = funds[(~funds['Papel'].isin(bancos))&(~funds['Papel'].isin(seguros))&(~funds['Papel'].isin(util))]
b_print(
    funds[~funds['Papel'].isin(bancos)],10
)
          

Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,Rank_earnings_yield,Rank_return_on_capital,Rank_Final,prefixo
PSSA3,39.99,9.71,1.93,0.726,3.86,0.559,10.51,0.88,-4.42,0.49,0.48,82.56,7.59,1.10,84.44,19.84,88090700.0,1.341890e+10,0.00,19.31,1.0,1.0,2.0,PSSA
PLPL3,9.88,5.94,2.39,0.788,10.18,0.816,1.22,2.95,3.47,2.80,2.71,26.71,14.58,4.42,44.22,40.32,12806200.0,8.414080e+08,0.70,28.06,6.0,2.0,8.0,PLPL
WIZC3,5.97,5.31,1.59,0.756,4.12,0.382,-6.13,1.49,-1.34,1.63,1.36,50.82,24.55,0.77,31.37,29.99,2972500.0,5.991930e+08,0.57,13.02,2.0,6.0,8.0,WIZC
PETR4,37.69,5.74,1.23,0.964,21.14,0.451,-42.02,2.16,-0.94,3.25,2.48,44.64,16.90,0.94,23.16,21.51,1123500000.0,3.936550e+11,0.82,16.88,7.0,17.0,24.0,PETR
CMIN3,5.46,7.74,3.35,1.743,14.24,0.869,4.11,5.44,-3.36,4.40,3.64,32.04,22.53,1.78,30.62,43.27,42437100.0,8.945330e+09,0.98,-2.60,19.0,8.0,27.0,CMIN
SYNE3,4.92,1.53,0.48,0.538,133.15,0.198,0.70,1.45,-5.44,1.77,1.52,37.24,44.92,11.34,16.74,31.25,7587810.0,1.573570e+09,0.54,21.71,3.0,32.0,35.0,SYNE
VALE3,54.17,5.10,1.18,1.165,9.88,0.508,-32.86,3.09,-1.28,3.74,3.11,37.69,22.87,0.91,18.51,23.14,1613950000.0,2.085550e+11,0.37,0.44,12.0,28.0,40.0,VALE
LEVE3,28.35,6.57,3.46,0.903,8.33,1.174,3.39,4.77,-70.80,5.42,4.82,18.93,13.74,2.16,33.91,52.72,10118400.0,1.109300e+09,0.92,16.79,35.0,5.0,40.0,LEVE
VLID3,23.84,5.52,1.16,0.881,7.46,0.691,2.01,4.58,5.05,4.71,3.75,19.22,15.91,2.87,20.16,20.92,11544900.0,1.689020e+09,0.36,1.64,23.0,22.0,45.0,VLID
LREN3,13.65,11.70,1.39,1.026,4.30,0.754,3.29,4.95,5.91,4.47,3.15,20.74,8.77,1.64,19.34,11.89,216431000.0,1.039470e+10,0.11,16.64,20.0,25.0,45.0,LREN


#### sem seguradora

In [11]:
# funds = funds[(~funds['Papel'].isin(bancos))&(~funds['Papel'].isin(seguros))&(~funds['Papel'].isin(util))]
b_print(
    funds[~funds['Papel'].isin(seguros)],10
)

Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,Rank_earnings_yield,Rank_return_on_capital,Rank_Final,prefixo
PLPL3,9.88,5.94,2.39,0.788,10.18,0.816,1.22,2.95,3.47,2.80,2.71,26.71,14.58,4.42,44.22,40.32,12806200.0,8.414080e+08,0.70,28.06,6.0,2.0,8.0,PLPL
PETR4,37.69,5.74,1.23,0.964,21.14,0.451,-42.02,2.16,-0.94,3.25,2.48,44.64,16.90,0.94,23.16,21.51,1123500000.0,3.936550e+11,0.82,16.88,7.0,17.0,24.0,PETR
CMIN3,5.46,7.74,3.35,1.743,14.24,0.869,4.11,5.44,-3.36,4.40,3.64,32.04,22.53,1.78,30.62,43.27,42437100.0,8.945330e+09,0.98,-2.60,19.0,8.0,27.0,CMIN
SYNE3,4.92,1.53,0.48,0.538,133.15,0.198,0.70,1.45,-5.44,1.77,1.52,37.24,44.92,11.34,16.74,31.25,7587810.0,1.573570e+09,0.54,21.71,3.0,32.0,35.0,SYNE
VALE3,54.17,5.10,1.18,1.165,9.88,0.508,-32.86,3.09,-1.28,3.74,3.11,37.69,22.87,0.91,18.51,23.14,1613950000.0,2.085550e+11,0.37,0.44,12.0,28.0,40.0,VALE
LEVE3,28.35,6.57,3.46,0.903,8.33,1.174,3.39,4.77,-70.80,5.42,4.82,18.93,13.74,2.16,33.91,52.72,10118400.0,1.109300e+09,0.92,16.79,35.0,5.0,40.0,LEVE
VLID3,23.84,5.52,1.16,0.881,7.46,0.691,2.01,4.58,5.05,4.71,3.75,19.22,15.91,2.87,20.16,20.92,11544900.0,1.689020e+09,0.36,1.64,23.0,22.0,45.0,VLID
LREN3,13.65,11.70,1.39,1.026,4.30,0.754,3.29,4.95,5.91,4.47,3.15,20.74,8.77,1.64,19.34,11.89,216431000.0,1.039470e+10,0.11,16.64,20.0,25.0,45.0,LREN
SBSP3,94.57,6.93,1.71,1.817,1.52,0.849,64.89,4.02,-2.15,5.28,4.47,45.19,26.23,1.14,22.27,24.61,337626000.0,3.790900e+10,0.63,17.74,27.0,19.0,46.0,SBSP
ASAI3,6.74,14.33,1.82,0.126,1.38,0.208,-3.72,1.77,-0.39,4.21,3.21,7.15,0.88,0.86,17.82,12.73,130493000.0,4.995000e+09,3.33,40.13,17.0,29.0,46.0,ASAI


#### sem utilidade publica

In [12]:
# funds = funds[(~funds['Papel'].isin(bancos))&(~funds['Papel'].isin(seguros))&(~funds['Papel'].isin(util))]
b_print(
    funds[~funds['Papel'].isin(util)],10
)

Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,Rank_earnings_yield,Rank_return_on_capital,Rank_Final,prefixo
PSSA3,39.99,9.71,1.93,0.726,3.86,0.559,10.51,0.88,-4.42,0.49,0.48,82.56,7.59,1.10,84.44,19.84,88090700.0,1.341890e+10,0.00,19.31,1.0,1.0,2.0,PSSA
PLPL3,9.88,5.94,2.39,0.788,10.18,0.816,1.22,2.95,3.47,2.80,2.71,26.71,14.58,4.42,44.22,40.32,12806200.0,8.414080e+08,0.70,28.06,6.0,2.0,8.0,PLPL
WIZC3,5.97,5.31,1.59,0.756,4.12,0.382,-6.13,1.49,-1.34,1.63,1.36,50.82,24.55,0.77,31.37,29.99,2972500.0,5.991930e+08,0.57,13.02,2.0,6.0,8.0,WIZC
PETR4,37.69,5.74,1.23,0.964,21.14,0.451,-42.02,2.16,-0.94,3.25,2.48,44.64,16.90,0.94,23.16,21.51,1123500000.0,3.936550e+11,0.82,16.88,7.0,17.0,24.0,PETR
CMIN3,5.46,7.74,3.35,1.743,14.24,0.869,4.11,5.44,-3.36,4.40,3.64,32.04,22.53,1.78,30.62,43.27,42437100.0,8.945330e+09,0.98,-2.60,19.0,8.0,27.0,CMIN
SYNE3,4.92,1.53,0.48,0.538,133.15,0.198,0.70,1.45,-5.44,1.77,1.52,37.24,44.92,11.34,16.74,31.25,7587810.0,1.573570e+09,0.54,21.71,3.0,32.0,35.0,SYNE
VALE3,54.17,5.10,1.18,1.165,9.88,0.508,-32.86,3.09,-1.28,3.74,3.11,37.69,22.87,0.91,18.51,23.14,1613950000.0,2.085550e+11,0.37,0.44,12.0,28.0,40.0,VALE
LEVE3,28.35,6.57,3.46,0.903,8.33,1.174,3.39,4.77,-70.80,5.42,4.82,18.93,13.74,2.16,33.91,52.72,10118400.0,1.109300e+09,0.92,16.79,35.0,5.0,40.0,LEVE
VLID3,23.84,5.52,1.16,0.881,7.46,0.691,2.01,4.58,5.05,4.71,3.75,19.22,15.91,2.87,20.16,20.92,11544900.0,1.689020e+09,0.36,1.64,23.0,22.0,45.0,VLID
LREN3,13.65,11.70,1.39,1.026,4.30,0.754,3.29,4.95,5.91,4.47,3.15,20.74,8.77,1.64,19.34,11.89,216431000.0,1.039470e+10,0.11,16.64,20.0,25.0,45.0,LREN


#### sem as 3 classes

In [13]:
b_print(
    funds[(~funds['Papel'].isin(bancos))&(~funds['Papel'].isin(seguros))&(~funds['Papel'].isin(util))],
    10
)

Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,Rank_earnings_yield,Rank_return_on_capital,Rank_Final,prefixo
PLPL3,9.88,5.94,2.39,0.788,10.18,0.816,1.22,2.95,3.47,2.80,2.71,26.71,14.58,4.42,44.22,40.32,12806200.0,8.414080e+08,0.70,28.06,6.0,2.0,8.0,PLPL
PETR4,37.69,5.74,1.23,0.964,21.14,0.451,-42.02,2.16,-0.94,3.25,2.48,44.64,16.90,0.94,23.16,21.51,1123500000.0,3.936550e+11,0.82,16.88,7.0,17.0,24.0,PETR
CMIN3,5.46,7.74,3.35,1.743,14.24,0.869,4.11,5.44,-3.36,4.40,3.64,32.04,22.53,1.78,30.62,43.27,42437100.0,8.945330e+09,0.98,-2.60,19.0,8.0,27.0,CMIN
SYNE3,4.92,1.53,0.48,0.538,133.15,0.198,0.70,1.45,-5.44,1.77,1.52,37.24,44.92,11.34,16.74,31.25,7587810.0,1.573570e+09,0.54,21.71,3.0,32.0,35.0,SYNE
VALE3,54.17,5.10,1.18,1.165,9.88,0.508,-32.86,3.09,-1.28,3.74,3.11,37.69,22.87,0.91,18.51,23.14,1613950000.0,2.085550e+11,0.37,0.44,12.0,28.0,40.0,VALE
LEVE3,28.35,6.57,3.46,0.903,8.33,1.174,3.39,4.77,-70.80,5.42,4.82,18.93,13.74,2.16,33.91,52.72,10118400.0,1.109300e+09,0.92,16.79,35.0,5.0,40.0,LEVE
VLID3,23.84,5.52,1.16,0.881,7.46,0.691,2.01,4.58,5.05,4.71,3.75,19.22,15.91,2.87,20.16,20.92,11544900.0,1.689020e+09,0.36,1.64,23.0,22.0,45.0,VLID
LREN3,13.65,11.70,1.39,1.026,4.30,0.754,3.29,4.95,5.91,4.47,3.15,20.74,8.77,1.64,19.34,11.89,216431000.0,1.039470e+10,0.11,16.64,20.0,25.0,45.0,LREN
ASAI3,6.74,14.33,1.82,0.126,1.38,0.208,-3.72,1.77,-0.39,4.21,3.21,7.15,0.88,0.86,17.82,12.73,130493000.0,4.995000e+09,3.33,40.13,17.0,29.0,46.0,ASAI
KEPL3,10.31,7.36,2.46,1.123,8.19,1.195,3.23,6.13,6.57,5.66,5.02,18.34,15.27,2.14,30.92,33.38,7437010.0,7.544800e+08,0.42,26.77,41.0,7.0,48.0,KEPL


outros filtros

In [14]:
funds2 =  funds[funds['P/L'] > 0]                # sem prejuizo atual
funds2 =  funds2[(funds['P/L'] < 30)]           # não estar excessivamente cara
funds2 =  funds2[funds['Dív.Brut/ Patrim.'] < 3.5]  # endividamento sob controle
funds2 =  funds2[funds['Cresc. Rec.5a'] > 0]    # crescimento nos ultimos 5 anos

In [15]:
b_print(
    funds2[(~funds2['Papel'].isin(bancos))&(~funds2['Papel'].isin(seguros))&(~funds2['Papel'].isin(util))],
    10
)

Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,Rank_earnings_yield,Rank_return_on_capital,Rank_Final,prefixo
PLPL3,9.88,5.94,2.39,0.788,10.18,0.816,1.22,2.95,3.47,2.80,2.71,26.71,14.58,4.42,44.22,40.32,12806200.0,8.414080e+08,0.70,28.06,6.0,2.0,8.0,PLPL
PETR4,37.69,5.74,1.23,0.964,21.14,0.451,-42.02,2.16,-0.94,3.25,2.48,44.64,16.90,0.94,23.16,21.51,1123500000.0,3.936550e+11,0.82,16.88,7.0,17.0,24.0,PETR
SYNE3,4.92,1.53,0.48,0.538,133.15,0.198,0.70,1.45,-5.44,1.77,1.52,37.24,44.92,11.34,16.74,31.25,7587810.0,1.573570e+09,0.54,21.71,3.0,32.0,35.0,SYNE
VALE3,54.17,5.10,1.18,1.165,9.88,0.508,-32.86,3.09,-1.28,3.74,3.11,37.69,22.87,0.91,18.51,23.14,1613950000.0,2.085550e+11,0.37,0.44,12.0,28.0,40.0,VALE
LEVE3,28.35,6.57,3.46,0.903,8.33,1.174,3.39,4.77,-70.80,5.42,4.82,18.93,13.74,2.16,33.91,52.72,10118400.0,1.109300e+09,0.92,16.79,35.0,5.0,40.0,LEVE
VLID3,23.84,5.52,1.16,0.881,7.46,0.691,2.01,4.58,5.05,4.71,3.75,19.22,15.91,2.87,20.16,20.92,11544900.0,1.689020e+09,0.36,1.64,23.0,22.0,45.0,VLID
LREN3,13.65,11.70,1.39,1.026,4.30,0.754,3.29,4.95,5.91,4.47,3.15,20.74,8.77,1.64,19.34,11.89,216431000.0,1.039470e+10,0.11,16.64,20.0,25.0,45.0,LREN
ASAI3,6.74,14.33,1.82,0.126,1.38,0.208,-3.72,1.77,-0.39,4.21,3.21,7.15,0.88,0.86,17.82,12.73,130493000.0,4.995000e+09,3.33,40.13,17.0,29.0,46.0,ASAI
KEPL3,10.31,7.36,2.46,1.123,8.19,1.195,3.23,6.13,6.57,5.66,5.02,18.34,15.27,2.14,30.92,33.38,7437010.0,7.544800e+08,0.42,26.77,41.0,7.0,48.0,KEPL
CEAB3,9.79,8.24,0.95,0.409,3.52,0.344,3.42,3.62,-2.01,3.96,2.13,11.29,4.96,1.27,14.49,11.51,42546200.0,3.180770e+09,0.45,15.36,15.0,48.0,63.0,CEAB


retirar BDR

In [16]:
# Padrão a ser buscado XYZW3[2,3,4,5]
padrao = r'[A-Z]{4}3[2-5]'

# Filtrando as linhas onde o padrão ocorre na coluna 'Papel'
funds[funds['Papel'].str.contains(padrao)]


,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,...,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,Rank_earnings_yield,Rank_return_on_capital,Rank_Final
19,AURA33,24.71,-19.49,4.28,1.892,3.09,1.032,26.31,5.59,-2.53,...,1.13,26.76,-21.94,5258660.0,1.255340e+09,1.48,13.14,58.0,14.0,72.0


A fórmula:
1. Estabeleça uma capitalização de mercado mínima.
2. Exclua ações de serviços públicos e financeiras  (ou seja, fundos mútuos, bancos e companhias de seguros).
3. Excluir empresas estrangeiras (ADRs)
4. Determine o rendimento dos lucros da empresa = EBIT/EV
5. Determine o retorno sobre o capital da empresa = EBIT/ (Ativo Fixo Líquido + Capital de Giro).
6. Classifique todas as empresas acima da capitalização de mercado escolhida pelo maior rendimento de lucros e maior retorno sobre o capital (classificado como porcentagens)
7. Invista em 20 a 30 empresas com melhor classificação, acumulando 2 a 3 posições por mês durante um período de 12 meses
8. Reequilibre o portfólio uma vez por ano, vendendo os perdedores uma semana antes do ano e os vencedores uma semana depois do ano.
9. Continuar por um período de longo prazo (5 a 10+ anos)

https://en.wikipedia.org/wiki/Magic_formula_investing

O retorno sobre o capital de Greenblatt difere de um valor típico de ROE ou ROIC. Dentro da Fórmula Mágica, o retorno sobre o capital de uma empresa é medido como EBIT/capital tangível empregado. Em outras palavras, estamos tentando encontrar os custos tangíveis para o negócio na geração dos lucros reportados dentro do período, onde o capital tangível empregado é definido mais precisamente como Capital Circulante Líquido mais Ativos Fixos Líquidos.

O capital de giro líquido é simplesmente o total do ativo circulante menos o passivo circulante, com um ajuste para remover dívidas com juros de curto prazo do passivo circulante e outro para remover o excesso de caixa. Greenblatt não oferece detalhes sobre como o excesso de caixa deve ser considerado, mas muitas vezes é calculado com base em uma porcentagem do caixa necessário em relação às vendas geradas em um período.

In [17]:
data_funds.loc['INEP4']

Cotação              1.400000e+00
P/L                  1.200000e-01
P/VP                -5.000000e-02
PSR                  1.446900e+01
Div.Yield            0.000000e+00
P/Ativo              6.100000e-02
P/Cap.Giro          -6.000000e-02
P/EBIT              -9.900000e-01
P/Ativ Circ.Liq     -3.000000e-02
EV/EBIT             -1.555000e+01
EV/EBITDA           -1.738000e+01
Mrg Ebit            -1.461870e+03
Mrg. Líq.            1.181240e+04
Liq. Corr.           8.000000e-02
ROIC                -6.530000e+00
ROE                 -3.690000e+01
Liq.2meses           6.586070e+04
Patrim. Líq         -1.377160e+09
Dív.Brut/ Patrim.   -6.600000e-01
Cresc. Rec.5a       -9.090000e+00
Name: INEP4, dtype: float64